In [1]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel

print("Libraries installed and ready!")

Libraries installed and ready!


In [2]:
import os

print(os.getcwd())
print(os.listdir(".."))

c:\Users\HP\OneDrive\Desktop\fake-news-detection\Dataset\notebook
['Fake.csv', 'news.csv', 'notebook', 'True.csv']


In [3]:
print(os.listdir(".."))

['Fake.csv', 'news.csv', 'notebook', 'True.csv']


In [4]:
fake_df = pd.read_csv("../Fake.csv")
true_df = pd.read_csv("../True.csv")

print("Fake shape:", fake_df.shape)
print("True shape:", true_df.shape)

Fake shape: (23481, 4)
True shape: (21417, 4)


In [5]:
fake_df["label"] = 0
true_df["label"] = 1

fake_df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [6]:
df = pd.concat([fake_df, true_df])
df = df.sample(frac=1).reset_index(drop=True)

df.head()

,title,text,subject,date,label
0,Gabby Giffords Has Made Her Choice For Who Sh...,The adversity that former Congresswoman Gabby ...,News,"January 10, 2016",0
1,"U.S. Senators McCain, Graham: Order may help r...",WASHINGTON (Reuters) - Republican U.S. senator...,politicsNews,"January 29, 2017",1
2,Elizabeth Warren Gives Trump Bad News After H...,Hillary Clinton announced her choice of Virgin...,News,"July 23, 2016",0
3,Trump Started A War With McConnell And The Ol...,Donald Trump just LOVES attacking members of h...,News,"August 10, 2017",0
4,Suicide bombers attack power station north of ...,"TIKRIT, Iraq (Reuters) - Suicide bombers struc...",worldnews,"September 2, 2017",1


In [7]:
# Combine title + text
df["content"] = df["title"] + " " + df["text"]

# Keep only needed columns
df = df[["content", "label"]]

df.head()

,content,label
0,Gabby Giffords Has Made Her Choice For Who Sh...,0
1,"U.S. Senators McCain, Graham: Order may help r...",1
2,Elizabeth Warren Gives Trump Bad News After H...,0
3,Trump Started A War With McConnell And The Ol...,0
4,Suicide bombers attack power station north of ...,1


In [8]:
from transformers import BertTokenizer

# Load pretrained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

print("Tokenizer loaded successfully!")

Tokenizer loaded successfully!


In [9]:
# Take one sample
sample_text = df["content"].iloc[0]

# Tokenize it
encoding = tokenizer(
    sample_text,
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

print(encoding)

{'input_ids': tensor([[  101, 27183, 29360,  2015,  2038,  2081,  2014,  3601,  2005,  2040,
          2016,  4122,  2004,  1996,  2279,  2343,  1996,  4748, 14028,  3012,
          2008,  2280,  3519, 10169, 27183, 29360,  2015,  1006,  1040,  1011,
         17207,  1007,  2038,  2018,  2000,  9462,  2003,  2498,  2460,  1997,
          3052, 17904,  1012,  2044,  2108,  2915,  1999,  1996,  2132,  2076,
          1037,  3113,  1998, 17021,  2007,  2014, 24355,  2274,  2086,  3283,
          1010,  2016,  2038,  2081,  1037, 12845,  2008,  2071,  2404,  2014,
          2012,  5394,  3570,  1012,  2016,  1055,  1037,  5394,  1010,  2138,
          2025,  2069,  2038,  2016,  2081,  1037, 19828,  7233,  1010,  2021,
          2016,  2038,  4056,  2014,  2166,  2000,  2667,  2000,  2437,  2469,
          2053,  2028,  2038,  2000, 18094,  1996,  3255,  1997,  3282,  4808,
          1012,  2083,  2014,  3029,  4841,  2005,  3625,  7300,  1010,  4077,
          2014,  3129,  3394, 19748,  

In [10]:
from transformers import BertForSequenceClassification

# Load pretrained BERT for classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

print("Model loaded successfully!")

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Model loaded successfully!


In [11]:
import torch

# Take one sample text
sample_text = df["content"].iloc[0]
true_label = df["label"].iloc[0]

# Tokenize
inputs = tokenizer(
    sample_text,
    padding="max_length",
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Turn off gradient calculation (faster)
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted class
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

print("True label:", true_label)
print("Predicted label:", predicted_class)

True label: 0
Predicted label: 0


In [12]:
# Take small subset for training (2000 samples)
df_small = df.sample(n=2000, random_state=42).reset_index(drop=True)

print(df_small.shape)

(2000, 2)


In [13]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_small["content"].tolist(),
    df_small["label"].tolist(),
    test_size=0.2,
    random_state=42
)

print("Train size:", len(train_texts))
print("Validation size:", len(val_texts))

Train size: 1600
Validation size: 400


In [14]:
# Tokenize training data
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)

# Tokenize validation data
val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128
)

print("Tokenization complete")

Tokenization complete


In [15]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

print("Dataset ready")

Dataset ready


In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

print("Training arguments ready")

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Training arguments ready


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

print("Trainer ready")

Trainer ready


In [18]:
trainer.train()

C:\Users\HP\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Step,Training Loss
10,0.726631
20,0.653329
30,0.556876
40,0.433397
50,0.210014
60,0.089493
70,0.025180
80,0.005808
90,0.001868
100,0.001074


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=200, training_loss=0.13541254382464105, metrics={'train_runtime': 1570.8696, 'train_samples_per_second': 1.019, 'train_steps_per_second': 0.127, 'total_flos': 105244422144000.0, 'train_loss': 0.13541254382464105, 'epoch': 1.0})

In [19]:
trainer.evaluate()

C:\Users\HP\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


{'eval_loss': 0.00023887572751846164,
 'eval_runtime': 108.5875,
 'eval_samples_per_second': 3.684,
 'eval_steps_per_second': 0.46,
 'epoch': 1.0}

In [20]:
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [22]:
def predict_news(text):
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    if predicted_class == 0:
        return "Fake News"
    else:
        return "Real News"

In [23]:
test_text = "The government announced a new economic reform policy today."
print(predict_news(test_text))


Fake News


In [24]:
test_text = "Aliens have taken control of the parliament according to secret sources."
print(predict_news(test_text))

Fake News


In [25]:
model.save_pretrained("./fake_news_model")
tokenizer.save_pretrained("./fake_news_model")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('./fake_news_model\\tokenizer_config.json',
 './fake_news_model\\tokenizer.json')

In [26]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Get predictions on validation set
predictions = trainer.predict(val_dataset)

pred_labels = np.argmax(predictions.predictions, axis=1)

accuracy = accuracy_score(val_labels, pred_labels)

print("Validation Accuracy:", accuracy)
print("\nClassification Report:\n")
print(classification_report(val_labels, pred_labels))

Validation Accuracy: 1.0

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       206
           1       1.00      1.00      1.00       194

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



In [27]:
trainer.save_model("fake_news_model")
tokenizer.save_pretrained("fake_news_model")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('fake_news_model\\tokenizer_config.json', 'fake_news_model\\tokenizer.json')

In [28]:
test_text = "Government announces new economic reform policy"
predict_news(test_text)

'Fake News'

In [29]:
test_text = """
WASHINGTON (Reuters) - The government announced new economic reforms on Monday aimed at improving employment and strengthening the national economy, officials said.
"""

predict_news(test_text)

'Real News'

In [30]:
num_train_epochs = 3

In [31]:
test_cases = [
    # Likely REAL (Reuters style)
    """WASHINGTON (Reuters) - The U.S. government announced new economic reforms on Monday aimed at boosting employment and strengthening the national economy, officials said.""",

    """LONDON (Reuters) - Prime Minister stated that new healthcare policies will be introduced next year to improve public services and reduce costs.""",

    """NEW YORK (Reuters) - The Federal Reserve confirmed that interest rates will remain unchanged following the latest economic review meeting.""" ,

    # Likely FAKE (sensational style)
    "Aliens officially land in Texas and government confirms secret treaty.",
    
    "Scientists prove the earth is flat and NASA hides the truth.",
    
    "Breaking: Secret documents reveal shocking conspiracy behind global elections."
]

for i, text in enumerate(test_cases):
    prediction = predict_news(text)
    print(f"Test Case {i+1}: {prediction}")
    print("-" * 50)

Test Case 1: Real News
--------------------------------------------------
Test Case 2: Real News
--------------------------------------------------
Test Case 3: Real News
--------------------------------------------------
Test Case 4: Fake News
--------------------------------------------------
Test Case 5: Fake News
--------------------------------------------------
Test Case 6: Fake News
--------------------------------------------------
